In [ ]:
#Comparison_of_Crossover_Methods
#Researchers: Mohammad Mahdi Ghaderi  &  Leily Khazraji

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 20 city
data = {
    'City': ['Tehran', 'Isfahan', 'Tabriz', 'Shiraz', 'Mashhad',
             'Kermanshah', 'Yazd', 'Karaj', 'Ahvaz', 'Qom',
             'Urmia', 'Arak', 'Kerman', 'Zanjan', 'Sari',
             'Gorgan', 'Bandar Abbas', 'Birjand', 'Sabzevar', 'Bojnurd'],
    'Latitude': [35.6892, 32.6546, 38.0962, 29.5918, 36.2605,
                 34.3293, 31.8974, 35.8325, 31.3193, 34.639,
                 37.5505, 34.0944, 30.2835, 36.6732, 36.6751,
                 36.6975, 27.1884, 32.8716, 33.0565, 37.4875],
    'Longitude': [51.3890, 51.6570, 46.2913, 52.5836, 59.5443,
                  47.1167, 54.3660, 51.9792, 48.6692, 50.8764,
                  45.9773, 49.6957, 57.0786, 48.5014, 53.0204,
                  54.0172, 56.2167, 59.2253, 57.6775, 57.1847]
}

In [ ]:
iran_df = pd.DataFrame(data)

In [ ]:
points = list(zip(iran_df['Longitude'], iran_df['Latitude']))

In [ ]:
# genetic algorithm
class GeneticAlgorithm:
    def __init__(self, points, population_size=100, generations=500, mutation_rate=0.01, crossover_method='PMX'):
        self.points = points
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_method = crossover_method
        self.population = self.initialize_population()

    def initialize_population(self):
        return [np.random.permutation(len(self.points)) for _ in range(self.population_size)]

    def calculate_distance(self, tour):
        distance = 0
        for i in range(len(tour)):
            point1 = self.points[tour[i]]
            point2 = self.points[tour[(i + 1) % len(tour)]]
            distance += np.linalg.norm(np.array(point1) - np.array(point2))
        return distance

    def select_parents(self):
        fitness = [1 / self.calculate_distance(tour) for tour in self.population]
        total_fitness = sum(fitness)

        if total_fitness == 0:
            raise ValueError("Total fitness is zero. Something went wrong with the distances.")

        probabilities = [f / total_fitness for f in fitness]

        population_indices = np.arange(len(self.population))

        return np.random.choice(population_indices, size=2, p=probabilities)

    def crossover(self, parent1, parent2):
        if self.crossover_method == 'PMX':
            return self.pmx_crossover(parent1, parent2)
        elif self.crossover_method == 'OX':
            return self.ox_crossover(parent1, parent2)
        elif self.crossover_method == 'CX':
            return self.cx_crossover(parent1, parent2)
        elif self.crossover_method == 'UX':
            return self.ux_crossover(parent1, parent2)
        else:
            raise ValueError("Invalid crossover method")

    def pmx_crossover(self, parent1, parent2):
        start, end = sorted(np.random.choice(len(parent1), 2, replace=False))
        child = [None] * len(parent1)
        child[start:end + 1] = parent1[start:end + 1]

        current_position = (end + 1) % len(parent1)
        for gene in parent2:
            if gene not in child:
                child[current_position] = gene
                current_position = (current_position + 1) % len(parent1)
        return np.array(child)

    def ox_crossover(self, parent1, parent2):
        start, end = sorted(np.random.choice(len(parent1), 2, replace=False))
        child = [None] * len(parent1)
        child[start:end + 1] = parent1[start:end + 1]

        parent2_idx = end + 1
        child_idx = end + 1

        while None in child:
            if parent2[parent2_idx % len(parent1)] not in child:
                child[child_idx % len(parent1)] = parent2[parent2_idx % len(parent1)]
                child_idx += 1
            parent2_idx += 1
        return np.array(child)

    def cx_crossover(self, parent1, parent2):
        child = [-1] * len(parent1)
        cycle_start = 0
        while -1 in child:
            if child[cycle_start] != -1:
                cycle_start = child.index(-1)
            gene = parent1[cycle_start]
            while True:
                child[cycle_start] = gene
                gene = parent2[cycle_start]
                cycle_start = parent1.tolist().index(gene)
                if gene == parent1[child.index(-1)]:
                    break
        return np.array(child)

    def ux_crossover(self, parent1, parent2):
        child = np.copy(parent1)
        mask = np.random.rand(len(parent1)) < 0.5
        for i in range(len(parent1)):
            if mask[i]:
                child[i] = parent2[i]
        return np.unique(child, return_index=True)[1]

    def mutate(self, tour):
        if np.random.rand() < self.mutation_rate:
            idx1, idx2 = np.random.choice(len(tour), 2, replace=False)
            tour[idx1], tour[idx2] = tour[idx2], tour[idx1]

    def run(self):
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.population_size):
                parent_indices = self.select_parents()
                parent1 = self.population[parent_indices[0]]
                parent2 = self.population[parent_indices[1]]
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)

            self.population = new_population

        best_tour = min(self.population, key=self.calculate_distance)
        best_distance = self.calculate_distance(best_tour)
        return best_tour, best_distance

In [ ]:
# Testing each crossover method
results = {}
methods = ['PMX', 'OX', 'CX']

In [ ]:
for method in methods:
    ga = GeneticAlgorithm(points, crossover_method=method, generations=500)
    best_tour, best_distance = ga.run()
    results[method] = {
        'Best Tour': [iran_df['City'].iloc[i] for i in best_tour],
        'Best Distance': best_distance
    }

In [ ]:
for method, result in results.items():
    print(f"Crossover Method: {method}")
    print(f"Best Tour: {result['Best Tour']}")
    print(f"Best Distance: {result['Best Distance']}\n")